In [2]:
from bs4 import BeautifulSoup
# import urllib
# import cfscrape
import pandas as pd
import re
from pprint import pprint
import cPickle as pickle

# for selenium
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# for mongodb
import pymongo

### load list of recap URLs 
Scraped in p01-scrape-dramabeans-urls.ipynb

In [15]:
recaplist = pickle.load(open('saveurls.p', 'rb'))
print len(recaplist)
recaplist[-5::]

5020


[u'http://www.dramabeans.com/2007/03/dal-jas-spring-episode-5/',
 u'http://www.dramabeans.com/2007/02/summary-dal-ja-episode-4/',
 u'http://www.dramabeans.com/2007/02/summary-dal-ja-episode-3/',
 u'http://www.dramabeans.com/2007/02/summary-dal-jas-spring-episode-2/',
 u'http://www.dramabeans.com/2007/02/summary-dal-jas-spring-episode-1/']

### define functions to extract recaps

In [3]:
def totext(input):
    '''takes input (list), cleans it up '''
    output = ''
    for line in input:
        output += ' '+ line.strip()
    return output.strip()

# print totext(recap)
# print totext(longcomment)

In [4]:
# define function to extract out comments on the page
def getcomment(soup, comment_all):
    '''takes in the page soup, existing list of dictionary of comments, returns updated list of dictionary of comments'''

    # extract out username & time comment is posted
    userinfo = soup.findAll('div', {'class':'des'})
    # extract out comments
    comments_page = soup.findAll('div', {'class':'content-des'})

#     print len(userinfo), userinfo[-1]
#     print len(comments_page), comments_page[-1]
#     print type(comment_all), len(comment_all), len(comments_page)

    for i in range(len(comments_page)):

        this_comment = {'Username': userinfo[i].h3.a.text,
                        'Time': pd.to_datetime(userinfo[i].h6.text),
                        'Comment': comments_page[i].text.replace('\n', ' ').strip()
                       }
        comment_all.append(this_comment)
#         print 'added!'

    return comment_all


In [16]:
# define a function to compile 1 document of recaps and comments
def createentry(source):
    '''takes first page of drama episode, returns title (collection name) & 
    dictionary entry for db'''

    # convert to bs
    soup = BeautifulSoup(source, 'html.parser')
#     assert 'Episode' in soup.title.text

    # prep dictionary entry
    entry = {}
    entry['url'] = url

    # get title & episode number
    info = soup.find('h3', attrs={'class': 'post-title-thumbnail'}).text
    try:
        title, episode = info.rsplit(': ',1)  
    except:
        title, episode = info.rsplit(', ',1)  
    if '(' in title:
        title = title.rsplit('(')[0].strip().title()
    else:
        title.strip().title()
    entry['Drama'] = title
    entry['Episode'] = episode.strip()

    # get author of recap
    author = soup.find('h4', attrs={'class': 'post-author'}).a.text
    entry['Recap_Author'] = author.strip()

    # get time of post
    try:
        time = pd.to_datetime(soup.find('time', attrs={'class':'entry-date published'}).get('datetime'))
    except:
        time = pd.to_datetime(soup.find('time', attrs={'class':'entry-date published updated'}).get('datetime'))
    entry['Recap_Time'] = time

    # extract number of comments
    numc = soup.find('div', attrs={'class': 'comment'}).a.text
    if ',' in numc:
        numc = int(numc.replace(',', ''))
    else:
        numc = int(numc)
    entry['Num_Comments'] = numc
    
    # extract average user rating
    try:
        rating = soup.find('span', attrs={'class': 'avg-rating'}).text.strip().split(' ')[-1]
        entry['User_Rating'] = float(rating)
    except:
        entry['User_Rating'] = None
    
    # extract recap and long comment
    para = soup.body.find('div', attrs={'class':'post-item'}).find_all('p')

    c = []
    r = []
    recap = []
    longcomment = []
    
    for i, k in enumerate(para):
    #     print i, k.text
        if 'COMMENTS' in k.text:
            c.append(i)
        if 'RELATED POSTS' in k.text:
            r.append(i)

        # capture only recap section
        if (c == []) and (r == []):
            recap.append(k.text)

        # capture only long comment
        elif (c != []) and (r == []):
            longcomment.append(k.text)

    # recap        
    entry['Recap'] = totext(recap)

    # author's comment
    comment_all = [{'Username': author, 
                     'Time':time,
                     'Comment': totext(longcomment).lstrip('COMMENTS')
                    }]

    # extract out current page's comments
    comment_all = getcomment(soup, comment_all)
    counter = len(comment_all)

    # check if there is another page of comments
    page = 2
    while page != 99:
        nexturl = url + 'comment-page-' + str(page)
#         print nexturl
        driver.get(nexturl)
        wait.until(EC.element_to_be_clickable((By.XPATH, '//input[@class="btn btn-primary"]')))
        sleep(1)   
        driver.execute_script("window.stop();")
        source = driver.page_source
        # convert to bs
        soup = BeautifulSoup(source, 'html.parser')

        # extract out current page's comments
        comment_now = getcomment(soup, comment_all)
        if len(comment_now) == counter:
            page = 99
        else:
            comment_all = comment_now
            page +=1
            counter= len(comment_all)
        print 'Page & comment count:', page, counter

    # put all comments into entry
    entry['Comments'] = comment_all
    
    return title.lower(), entry


### connect to MongoDB

Prepare to save all scraped information to no-SQL database MongoDB.

In [3]:
# "C:\Program Files\MongoDB\Server\3.6\bin\mongod.exe" --dbpath "D:\Documents\Heidi\mongodb\data"

# Connection to Mongo DB
try:
    client = pymongo.MongoClient()
    print "Hooray, we have connected to MongoDB successfully!"
except pymongo.errors.ConnectionFailure, e:
    print "Could not connect to MongoDB: %s" % e 

# connect to the dramabeans database
db = client.dramabeans_v2

Hooray, we have connected to MongoDB successfully!


### call functions, extract recaps and save into MongoDB database

Using the functions defined above and the Dramabeans recap URLs previously scraped, run Selenium and scrape the recap URLs, ingesting the information using BeautifulSoup. Save all the elements into the MongoDB database set up, in the following manner:
> 1 Drama per collection:
>> 1 Episode per document, with the following fields:
- User_Rating
- Episode
- Recap Author
- URL
- Number of comments
- Array of comments, containing the username, comment, and time comment posted
- Recap posted time
- Recap


In [19]:
# create a driver called driver
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"

driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver.exe", desired_capabilities=capa)
wait = WebDriverWait(driver, 150)

# loop through recap urls, extract text from each one
for i in range(len(recaplist)):
    url = recaplist[i]
    driver.get(url)
#     wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@id="link-comments"]')))
    wait.until(EC.element_to_be_clickable((By.XPATH, '//input[@class="btn btn-primary"]')))
    sleep(1)   
    driver.execute_script("window.stop();")
    source = driver.page_source

    # get entry for db
    title, entry = createentry(source)

    # input into db
    collection = db[title]
    collection.insert_one(entry)
    
    # print if successful
    print i, url

Page & comment count: 99 110
0 http://www.dramabeans.com/2017/01/saimdang-lights-diary-episode-1/
Page & comment count: 99 42
1 http://www.dramabeans.com/2017/01/chief-kim-episode-1/
Page & comment count: 99 67
2 http://www.dramabeans.com/2017/01/introverted-boss-episode-4/
Page & comment count: 99 91
3 http://www.dramabeans.com/2017/01/missing-9-episode-4/
Page & comment count: 99 49
4 http://www.dramabeans.com/2017/01/defendant-episode-2/
Page & comment count: 99 82
5 http://www.dramabeans.com/2017/01/missing-9-episode-3/
Page & comment count: 99 66
6 http://www.dramabeans.com/2017/01/introverted-boss-episode-3/
Page & comment count: 3 239
Page & comment count: 99 239
7 http://www.dramabeans.com/2017/01/legend-of-the-blue-sea-episode-20-final/
Page & comment count: 99 56
8 http://www.dramabeans.com/2017/01/defendant-episode-1/
Page & comment count: 99 102
9 http://www.dramabeans.com/2017/01/voice-episode-4/
Page & comment count: 99 123
10 http://www.dramabeans.com/2017/01/hwarang-epi

Page & comment count: 99 54
74 http://www.dramabeans.com/2016/12/solomons-perjury-episode-1/
Page & comment count: 99 11
75 http://www.dramabeans.com/2016/12/entourage-episode-14/
Page & comment count: 99 17
76 http://www.dramabeans.com/2016/12/entourage-episode-13/
Page & comment count: 3 564
Page & comment count: 4 587
Page & comment count: 99 587
77 http://www.dramabeans.com/2016/12/the-lonely-shining-goblin-episode-6/
Page & comment count: 3 414
Page & comment count: 99 414
78 http://www.dramabeans.com/2016/12/the-lonely-shining-goblin-episode-5/
Page & comment count: 99 52
79 http://www.dramabeans.com/2016/12/romantic-doctor-teacher-kim-episode-12/
Page & comment count: 3 259
Page & comment count: 99 259
80 http://www.dramabeans.com/2016/12/weightlifting-fairy-kim-bok-ju-episode-10/
Page & comment count: 3 347
Page & comment count: 99 347
81 http://www.dramabeans.com/2016/12/legend-of-the-blue-sea-episode-10/
Page & comment count: 99 45
82 http://www.dramabeans.com/2016/12/romanti

Page & comment count: 99 76
147 http://www.dramabeans.com/2016/11/the-man-living-in-our-house-episode-7/
Page & comment count: 99 111
148 http://www.dramabeans.com/2016/11/1-of-anything-episode-12/
Page & comment count: 99 122
149 http://www.dramabeans.com/2016/11/this-week-my-wife-will-have-an-affair-episode-5/
Page & comment count: 3 231
Page & comment count: 99 231
150 http://www.dramabeans.com/2016/11/the-k2-episode-16-final/
Page & comment count: 99 85
151 http://www.dramabeans.com/2016/11/on-the-way-to-the-airport-episode-16-final/
Page & comment count: 3 268
Page & comment count: 99 268
152 http://www.dramabeans.com/2016/11/jealousy-incarnate-episode-24-final/
Page & comment count: 99 36
153 http://www.dramabeans.com/2016/11/entourage-episode-3/
Page & comment count: 99 136
154 http://www.dramabeans.com/2016/11/the-k2-episode-15/
Page & comment count: 99 22
155 http://www.dramabeans.com/2016/11/on-the-way-to-the-airport-episode-15/
Page & comment count: 3 111
Page & comment coun

Page & comment count: 99 108
224 http://www.dramabeans.com/2016/10/drinking-solo-episode-14/
Page & comment count: 99 34
225 http://www.dramabeans.com/2016/10/fantastic-episode-13/
Page & comment count: 99 53
226 http://www.dramabeans.com/2016/10/woman-with-a-suitcase-episode-7/
Page & comment count: 3 757
Page & comment count: 4 974
Page & comment count: 5 1292
Page & comment count: 6 1497
Page & comment count: 7 1502
Page & comment count: 99 1502
227 http://www.dramabeans.com/2016/10/moonlight-drawn-by-clouds-episode-18-final/
Page & comment count: 3 855
Page & comment count: 4 1129
Page & comment count: 99 1129
228 http://www.dramabeans.com/2016/10/moon-lovers-scarlet-heart-ryeo-episode-16/
Page & comment count: 99 69
229 http://www.dramabeans.com/2016/10/drinking-solo-episode-13/
Page & comment count: 3 446
Page & comment count: 99 446
230 http://www.dramabeans.com/2016/10/moonlight-drawn-by-clouds-episode-17/
Page & comment count: 99 59
231 http://www.dramabeans.com/2016/10/1-of-a

Page & comment count: 4 675
Page & comment count: 5 676
Page & comment count: 99 676
294 http://www.dramabeans.com/2016/09/moon-lovers-scarlet-heart-ryeo-episode-10/
Page & comment count: 3 303
Page & comment count: 99 303
295 http://www.dramabeans.com/2016/09/the-k2-episode-2/
Page & comment count: 99 75
296 http://www.dramabeans.com/2016/09/on-the-way-to-the-airport-episode-2/
Page & comment count: 3 155
Page & comment count: 99 155
297 http://www.dramabeans.com/2016/09/jealousy-incarnate-episode-10/
Page & comment count: 3 326
Page & comment count: 99 326
298 http://www.dramabeans.com/2016/09/the-k2-episode-1/
Page & comment count: 99 92
299 http://www.dramabeans.com/2016/09/on-the-way-to-the-airport-episode-1/
Page & comment count: 3 188
Page & comment count: 99 188
300 https://www.dramabeans.com/2016/09/shopping-king-louis-episode-2/
Page & comment count: 99 118
301 http://www.dramabeans.com/2016/09/jealousy-incarnate-episode-9/
Page & comment count: 99 14
302 http://www.dramabean

...

Page & comment count: 99 21
3892 http://www.dramabeans.com/2009/05/return-of-iljimae-episode-17/
Page & comment count: 99 49
3893 http://www.dramabeans.com/2009/04/story-of-a-man-episode-7/
Page & comment count: 99 54
3894 http://www.dramabeans.com/2009/04/story-of-a-man-episode-6/
Page & comment count: 99 51
3895 http://www.dramabeans.com/2009/04/story-of-a-man-episode-5/
Page & comment count: 3 69
Page & comment count: 99 69
3896 http://www.dramabeans.com/2009/04/story-of-a-man-episodes-3-4/
Page & comment count: 99 20
3897 http://www.dramabeans.com/2009/04/return-of-iljimae-episode-16/
Page & comment count: 99 27
3898 http://www.dramabeans.com/2009/04/return-of-iljimae-episode-15/
Page & comment count: 99 32
3899 http://www.dramabeans.com/2009/04/return-of-iljimae-episode-14/
Page & comment count: 3 107
Page & comment count: 4 157
Page & comment count: 5 207
Page & comment count: 6 257
Page & comment count: 7 307
Page & comment count: 8 357
Page & comment count: 9 407
Page & comment

Page & comment count: 99 222
3943 http://www.dramabeans.com/2009/01/boys-before-flowers-episode-1-korean-hana-yori-dango/
Page & comment count: 99 46
3944 http://www.dramabeans.com/2009/01/a-stars-lover-episode-8/
Page & comment count: 99 28
3945 http://www.dramabeans.com/2009/01/a-stars-lover-episode-7/
Page & comment count: 99 24
3946 http://www.dramabeans.com/2008/12/a-stars-lover-episode-6/
Page & comment count: 99 22
3947 http://www.dramabeans.com/2008/12/a-stars-lover-episode-5/
Page & comment count: 99 26
3948 http://www.dramabeans.com/2008/12/a-stars-lover-episode-4/
Page & comment count: 99 34
3949 http://www.dramabeans.com/2008/12/a-stars-lover-episode-3/
Page & comment count: 99 50
3950 http://www.dramabeans.com/2008/12/a-stars-lover-episode-2/
Page & comment count: 3 55
Page & comment count: 99 55
3951 http://www.dramabeans.com/2008/12/a-stars-lover-episode-1/
Page & comment count: 3 62
Page & comment count: 99 62
3952 http://www.dramabeans.com/2008/12/the-world-they-live-i

Page & comment count: 99 11
4028 http://www.dramabeans.com/2008/07/strongest-chil-woo-episode-12/
Page & comment count: 99 9
4029 http://www.dramabeans.com/2008/07/strongest-chil-woo-episode-11/
Page & comment count: 99 11
4030 http://www.dramabeans.com/2008/07/my-sweet-seoul-episode-11/
Page & comment count: 99 29
4031 http://www.dramabeans.com/2008/07/my-sweet-seoul-episode-10/
Page & comment count: 99 19
4032 http://www.dramabeans.com/2008/07/strongest-chil-woo-episode-10/
Page & comment count: 99 13
4033 http://www.dramabeans.com/2008/07/gourmet-episodes-7-8/
Page & comment count: 99 10
4034 http://www.dramabeans.com/2008/07/strongest-chil-woo-episode-9/
Page & comment count: 99 21
4035 http://www.dramabeans.com/2008/07/my-sweet-seoul-episode-9/
Page & comment count: 99 13
4036 http://www.dramabeans.com/2008/07/my-sweet-seoul-episode-8/
Page & comment count: 99 18
4037 http://www.dramabeans.com/2008/07/gourmet-episodes-5-6/
Page & comment count: 99 11
4038 http://www.dramabeans.com

4108 http://www.dramabeans.com/2007/09/the-story-of-hyang-dan-episode-2-end/
Page & comment count: 99 34
4109 http://www.dramabeans.com/2007/08/romance-hunter-episodes-1-5/
Page & comment count: 99 34
4110 http://www.dramabeans.com/2007/07/bad-couple-episode-16-final/
Page & comment count: 99 15
4111 http://www.dramabeans.com/2007/07/bad-couple-episode-15/
Page & comment count: 99 26
4112 http://www.dramabeans.com/2007/07/bad-couple-episode-14/
Page & comment count: 99 33
4113 http://www.dramabeans.com/2007/07/air-city-episode-16-final/
Page & comment count: 99 18
4114 http://www.dramabeans.com/2007/07/bad-couple-episode-13/
Page & comment count: 99 22
4115 http://www.dramabeans.com/2007/07/bad-couple-episode-12/
Page & comment count: 99 14
4116 http://www.dramabeans.com/2007/07/air-city-episodes-14-15/
Page & comment count: 99 27
4117 http://www.dramabeans.com/2007/07/bad-couple-episode-11/
Page & comment count: 3 97
Page & comment count: 99 97
4118 http://www.dramabeans.com/2007/07/f

Page & comment count: 99 5
4193 http://www.dramabeans.com/2007/03/dal-jas-spring-episode-18/
Page & comment count: 99 9
4194 http://www.dramabeans.com/2007/03/dal-jas-spring-episode-17/
Page & comment count: 99 11
4195 http://www.dramabeans.com/2007/03/dal-jas-spring-episode-16/
Page & comment count: 99 10
4196 http://www.dramabeans.com/2007/03/dal-jas-spring-episode-15/
Page & comment count: 99 12
4197 http://www.dramabeans.com/2007/03/dal-jas-spring-episode-14/
Page & comment count: 99 11
4198 http://www.dramabeans.com/2007/03/dal-jas-spring-episode-13/
Page & comment count: 99 14
4199 http://www.dramabeans.com/2007/03/dal-jas-spring-episode-6/
Page & comment count: 99 13
4200 http://www.dramabeans.com/2007/03/dal-jas-spring-episode-5/
Page & comment count: 99 14
4201 http://www.dramabeans.com/2007/02/summary-dal-ja-episode-4/
Page & comment count: 99 13
4202 http://www.dramabeans.com/2007/02/summary-dal-ja-episode-3/
Page & comment count: 99 7
4203 http://www.dramabeans.com/2007/02/